In [1]:
!pip install -U portalocker==2.8.2
!pip install datasets
!pip install evaluate
!pip install tqdm
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 23.5 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.3 MB/s eta 0:00:00


In [2]:
import torch
from datasets import load_dataset
from datasets import DatasetDict
from transformers import AutoTokenizer
import random
from transformers import DataCollatorWithPadding
from transformers import AutoModelForSequenceClassification
from transformers import AdamW, get_scheduler
from torch.utils.data import DataLoader
from tqdm.auto import tqdm
import evaluate


##### Reproducability configs

In [3]:
SEED = 42
torch.manual_seed(SEED)
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

### **Creating retain, forget, validation and test sets**

In [4]:
# create datasets for fine-tuning
raw_datasets = load_dataset('ag_news')

train_validation_datasets = raw_datasets['train'].train_test_split(test_size=0.1, seed=SEED)
train_forget_datasets = train_validation_datasets['train'].train_test_split(test_size=0.1, seed=SEED)
datasets = DatasetDict({
    'retain': train_forget_datasets['train'],
    'forget': train_forget_datasets['test'],
    'validation': train_validation_datasets['test'],
    'test': raw_datasets['test']
})
datasets

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/120000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7600 [00:00<?, ? examples/s]

DatasetDict({
    retain: Dataset({
        features: ['text', 'label'],
        num_rows: 97200
    })
    forget: Dataset({
        features: ['text', 'label'],
        num_rows: 10800
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 12000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 7600
    })
})

###**Data exploration**

##### uncomment commented lines to see data examples.

In [5]:
num_labels = datasets['retain'].features['label'].num_classes
id2label = {}
label2id = {}
for label_id,label in enumerate(datasets['retain'].features['label'].names):
    id2label[label_id] = label
    label2id[label] = label_id

print(f"NUM_LABELS: {num_labels}")
print(f"ID2LABEL: {id2label}")
print(f"LABEL2ID: {label2id}")


random.seed(SEED)

# get random integers in the range of 0 to train_dataset_length
EXAMPLE_INDICES = [random.randrange(len(datasets['retain'])) for _ in range(3)]
EXAMPLE_INDICES



NUM_LABELS: 4
ID2LABEL: {0: 'World', 1: 'Sports', 2: 'Business', 3: 'Sci/Tech'}
LABEL2ID: {'World': 0, 'Sports': 1, 'Business': 2, 'Sci/Tech': 3}


[83810, 14592, 3278]

### **Defining backbone and tokenizer**

In [6]:
backbone_type = 'distilbert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(backbone_type, use_fast=True)
tokenized_datasets = datasets.map(lambda x: tokenizer(x['text'], truncation=True), batched=True, remove_columns=['text'])
tokenized_datasets = tokenized_datasets.rename_column('label', 'labels')
tokenized_datasets.set_format(type='torch')

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/97200 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/10800 [00:00<?, ? examples/s]

Map:   0%|          | 0/12000 [00:00<?, ? examples/s]

Map:   0%|          | 0/7600 [00:00<?, ? examples/s]

#### uncomment to see what tokenized dataset look like

In [7]:
# @title
 # tokenized_datasets
#  for i in EXAMPLE_INDICES:
#     text = datasets['train']['text'][i]
#     tokenized_text = tokenized_datasets['train']['input_ids'][i]
#     label_id = tokenized_datasets['train']['labels'][i].item()
#     label = id2label[label_id]
#     print(f"          TEXT[{i}]: {text}")
#     print(f"TOKENIZED_TEXT[{i}]: {tokenized_text}")
#     print(f"         LABEL[{i}]: {label_id} ({label})")
#     print()

In [8]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

dataloaders = {
    'retain': None,
    'forget': None,
    'validation': None,
    'test': None,
}
for dataset_type in ['retain','forget', 'validation', 'test']:
    dataloaders[dataset_type] = DataLoader(
        dataset = tokenized_datasets[dataset_type],
        batch_size = 64,
        shuffle = True,
        collate_fn = data_collator,
    )

In [9]:
model = AutoModelForSequenceClassification.from_pretrained(backbone_type, num_labels=num_labels, id2label=id2label, label2id=label2id)

model.to(device)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [10]:

scheduler_name = 'linear'
optimizer = AdamW(model.parameters(), lr=5e-5, weight_decay=0, no_deprecation_warning=True)
num_training_epochs = 1
num_training_steps = num_training_epochs * len(dataloaders['retain'])
num_warmup_steps = 0
lr_scheduler = get_scheduler(
    name = scheduler_name,
    optimizer = optimizer,
    num_training_steps = num_training_steps,
    num_warmup_steps = num_warmup_steps,
)

print(f"           SCHEDULER NAME: {scheduler_name}")
print(f"                OPTIMIZER: {optimizer.__class__}")
print(f"NUMBER OF TRAINING EPOCHS: {num_training_epochs}")
print(f" NUMBER OF TRAINING STEPS: {num_training_steps}")

           SCHEDULER NAME: linear
                OPTIMIZER: <class 'transformers.optimization.AdamW'>
NUMBER OF TRAINING EPOCHS: 1
 NUMBER OF TRAINING STEPS: 1519


In [11]:
# the metrics for training and evaluation
accuracy_metric = evaluate.load('accuracy')
f1_metric = evaluate.load('f1')

In [12]:
def train(model, dataloader):

    model.train()
    loss = 0
    for batch in tqdm(dataloader):

        optimizer.zero_grad()


        batch = {k:v.to(device) for k,v in batch.items()}
        outputs = model(**batch)
        predictions = torch.argmax(outputs.logits, dim=-1)
        labels = batch['labels']

        # gradient descent
        outputs.loss.backward()
        optimizer.step()
        lr_scheduler.step()


        loss += outputs.loss.item()
        accuracy_metric.add_batch(predictions=predictions, references=labels)
        f1_metric.add_batch(predictions=predictions, references=labels)


    loss /= len(dataloader)
    accuracy = accuracy_metric.compute()
    f1 = f1_metric.compute(average='macro')
    return {'loss':loss, **accuracy, **f1}

In [13]:
def evaluate(model, dataloader):

    model.eval()
    loss = 0

    with torch.no_grad():
        for batch in tqdm(dataloader):

            batch = {k:v.to(device) for k,v in batch.items()}
            outputs = model(**batch)
            predictions = torch.argmax(outputs.logits, dim=-1)
            labels = batch['labels']


            loss += outputs.loss.item()
            accuracy_metric.add_batch(predictions=predictions, references=labels)
            f1_metric.add_batch(predictions=predictions, references=labels)


    loss /= len(dataloader)
    accuracy = accuracy_metric.compute()
    f1 = f1_metric.compute(average='macro')
    return {'loss':loss, **accuracy, **f1}

In [14]:
def grad_loss(inputs, labels):
  l = torch.nn.CrossEntropyLoss()
  return l(inputs, labels)

In [15]:
def train_with_neggrad(model, dataloader, loss_fn=None):

    model.train()
    loss = 0
    for batch in tqdm(dataloader):

        optimizer.zero_grad()

        batch = {k:v.to(device) for k,v in batch.items()}
        outputs = model(**batch)
        predictions = torch.argmax(outputs.logits, dim=-1)
        labels = batch['labels']
        logits = outputs[1]

        # gradient ascent

        loss = -1*grad_loss(logits, labels)
        loss.backward()
        unlearn_optimizer.step()
        unlearn_lr_scheduler.step()

        # accumulate metrics
        loss += outputs.loss.item()
        accuracy_metric.add_batch(predictions=predictions, references=labels)
        f1_metric.add_batch(predictions=predictions, references=labels)

    # return metrics
    loss /= len(dataloader)
    accuracy = accuracy_metric.compute()
    f1 = f1_metric.compute(average='macro')
    return {'loss':loss, **accuracy, **f1}

In [16]:
test_metrics = evaluate(model, dataloaders['test'])
print(f"TEST ACCURACY: {test_metrics['accuracy']:.5f}", end=" ; ")
print(f"F1 (MACRO): {test_metrics['f1']:.5f}")

  0%|          | 0/119 [00:00<?, ?it/s]

TEST ACCURACY: 0.23487 ; F1 (MACRO): 0.13023


In [17]:
for epoch in range(num_training_epochs):
    train_metrics = train(model, dataloaders['retain'])
    validation_metrics = evaluate(model, dataloaders['validation'])

    print(f"EPOCH {epoch+1}", end=" | ")
    print(f"TRAIN LOSS: {train_metrics['loss']:.5f}", end=" | ")
    print(f"VALIDATION LOSS: {validation_metrics['loss']:.5f}", end=" ; ")
    print(f"ACCURACY: {validation_metrics['accuracy']:.5f}", end=" ; ")
    print(f"F1 (MACRO): {validation_metrics['f1']:.5f}")

  0%|          | 0/1519 [00:00<?, ?it/s]

  0%|          | 0/188 [00:00<?, ?it/s]

EPOCH 1 | TRAIN LOSS: 0.22148 | VALIDATION LOSS: 0.17127 ; ACCURACY: 0.94325 ; F1 (MACRO): 0.94294


In [18]:
retain_metrics = evaluate(model, dataloaders['retain'])
print(f"retain ACCURACY: {retain_metrics['accuracy']:.5f}", end=" ; ")
print(f"F1 (MACRO): {retain_metrics['f1']:.5f}")

  0%|          | 0/1519 [00:00<?, ?it/s]

retain ACCURACY: 0.95736 ; F1 (MACRO): 0.95737


In [19]:
forget_metrics = evaluate(model, dataloaders['forget'])
print(f"TEST ACCURACY: {forget_metrics['accuracy']:.5f}", end=" ; ")
print(f"F1 (MACRO): {forget_metrics['f1']:.5f}")

  0%|          | 0/169 [00:00<?, ?it/s]

TEST ACCURACY: 0.94167 ; F1 (MACRO): 0.94161


In [20]:
test_metrics = evaluate(model, dataloaders['test'])
print(f"TEST ACCURACY: {test_metrics['accuracy']:.5f}", end=" ; ")
print(f"F1 (MACRO): {test_metrics['f1']:.5f}")

  0%|          | 0/119 [00:00<?, ?it/s]

TEST ACCURACY: 0.94066 ; F1 (MACRO): 0.94060


In [21]:
# let's use a linear scheduler with AdamW
scheduler_name = 'linear'
unlearn_optimizer = AdamW(model.parameters(), lr=5e-7, weight_decay=0, no_deprecation_warning=True)
num_training_epochs = 1
num_training_steps = num_training_epochs * len(dataloaders['forget'])
num_warmup_steps = 0
unlearn_lr_scheduler = get_scheduler(
    name = scheduler_name,
    optimizer = unlearn_optimizer,
    num_training_steps = num_training_steps,
    num_warmup_steps = num_warmup_steps,
)

print(f"           SCHEDULER NAME: {scheduler_name}")
print(f"                unlearn_OPTIMIZER: {optimizer.__class__}")
print(f"NUMBER OF TRAINING EPOCHS: {num_training_epochs}")
print(f" NUMBER OF TRAINING STEPS: {num_training_steps}")

           SCHEDULER NAME: linear
                unlearn_OPTIMIZER: <class 'transformers.optimization.AdamW'>
NUMBER OF TRAINING EPOCHS: 1
 NUMBER OF TRAINING STEPS: 169


## Unlearning with Catastrophic forgetting

In [22]:
unlearning_metrics  = train(model, dataloaders['retain'])

  0%|          | 0/1519 [00:00<?, ?it/s]

In [24]:
retain_metrics = evaluate(model, dataloaders['retain'])
print(f"retain ACCURACY: {retain_metrics['accuracy']:.5f}", end=" ; ")
print(f"F1 (MACRO): {retain_metrics['f1']:.5f}")

  0%|          | 0/1519 [00:00<?, ?it/s]

retain ACCURACY: 0.95736 ; F1 (MACRO): 0.95737


In [25]:
forget_metrics = evaluate(model, dataloaders['forget'])
print(f"TEST ACCURACY: {forget_metrics['accuracy']:.5f}", end=" ; ")
print(f"F1 (MACRO): {forget_metrics['f1']:.5f}")

  0%|          | 0/169 [00:00<?, ?it/s]

TEST ACCURACY: 0.94167 ; F1 (MACRO): 0.94161


In [26]:
test_metrics = evaluate(model, dataloaders['test'])
print(f"TEST ACCURACY: {test_metrics['accuracy']:.5f}", end=" ; ")
print(f"F1 (MACRO): {test_metrics['f1']:.5f}")

  0%|          | 0/119 [00:00<?, ?it/s]

TEST ACCURACY: 0.94066 ; F1 (MACRO): 0.94060
